<a href="https://colab.research.google.com/github/krmanu07/ML/blob/master/sheep_breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# kaggle datasets download -d divyansh22/sheep-breed-classification

In [3]:
!pip install -q kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"catratmanu","key":"589976db9d2232c6146be908287df238"}'}

In [5]:
!mkdir ~/.kaggle

In [7]:
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d divyansh22/sheep-breed-classification

 84% 9.00M/10.7M [00:00<00:00, 54.9MB/s]
100% 10.7M/10.7M [00:00<00:00, 63.4MB/s]


In [ ]:
!unzip '/content/sheep-breed-classification.zip' -d '/content/sheep-breed-classification'

In [29]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from skimage.transform import rotate
%matplotlib inline

In [12]:
dir_ = r'/content/sheep-breed-classification/SheepFaceImages'

In [13]:
types = [i for i in os.listdir(dir_)]

In [15]:
types

['White Suffolk', 'Poll Dorset', 'Suffolk', 'Marino']

In [59]:
types = ['White Suffolk', 'Poll Dorset', 'Suffolk', 'Marino']
images = []
labels = []
h, w = 50, 200
for i, breed in enumerate(types):
    img_path = dir_ + '/' + breed
    for j in os.listdir(img_path):
        img = cv2.imread(os.path.join(img_path, j), 0)
        for angle in range(3):
            new_img = (rotate(img, np.random.choice([-5, -4, -3, -2, -1, 1, 2, 3, 4, 5])) * 255).astype('uint8')
            new_img = cv2.resize(new_img, (w, h), cv2.INTER_CUBIC)
            new_img = np.array([new_img]).reshape(h, w, 1)
            images.append(new_img)
            label = [0] * len(types)
            label[i] = 1  # Set the corresponding breed index to 1
            labels.append(label)
        img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
        img = np.array([img]).reshape(h, w, 1)
        images.append(img)
        label = [0] * len(types)
        label[i] = 1  # Set the corresponding breed index to 1
        labels.append(label)

images = np.array(images)/255
labels = np.array(labels)
print(images.shape)
print(labels.shape)

(6720, 50, 200, 1)
(6720, 4)


In [58]:
labels[6719]

[0, 0, 0, 1]

In [60]:
import tensorflow as tf
from tensorflow.keras import layers as ly
from tensorflow.keras.models import Sequential

In [63]:
model = Sequential()
model.add(ly.Conv2D(32, (3,3), input_shape= (h,w, 1), activation = 'relu'))
model.add(ly.MaxPooling2D((2,2)))
model.add(ly.Conv2D(64, (3,3), activation = 'relu'))
model.add(ly.MaxPooling2D((2,2)))
model.add(ly.Conv2D(128, (3,3), activation = 'relu'))
model.add(ly.MaxPooling2D((3,3)))
model.add(ly.Flatten())
model.add(ly.Dense(32, activation = 'relu'))
model.add(ly.Dense(64, activation = 'relu'))
model.add(ly.Dense(4, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 48, 198, 32)       320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 99, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 22, 97, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 11, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 9, 46, 128)        73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 3, 15, 128)      

In [64]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
model.fit(images, labels, validation_split=0.15, shuffle = True, epochs = 40, batch_size = 32)

Epoch 1/40
179/179 [==============================] - 15s 17ms/step - loss: 0.1248 - accuracy: 0.5802 - val_loss: 0.2062 - val_accuracy: 0.3730
Epoch 2/40
179/179 [==============================] - 2s 12ms/step - loss: 0.0650 - accuracy: 0.8267 - val_loss: 0.2954 - val_accuracy: 0.1716
Epoch 3/40
179/179 [==============================] - 2s 11ms/step - loss: 0.0421 - accuracy: 0.8874 - val_loss: 0.1522 - val_accuracy: 0.5843
Epoch 4/40
179/179 [==============================] - 2s 11ms/step - loss: 0.0285 - accuracy: 0.9273 - val_loss: 0.2752 - val_accuracy: 0.2679
Epoch 5/40
179/179 [==============================] - 2s 11ms/step - loss: 0.0232 - accuracy: 0.9396 - val_loss: 0.1364 - val_accuracy: 0.6389
Epoch 6/40
179/179 [==============================] - 2s 11ms/step - loss: 0.0141 - accuracy: 0.9641 - val_loss: 0.1392 - val_accuracy: 0.6300
Epoch 7/40
179/179 [==============================] - 2s 13ms/step - loss: 0.0118 - accuracy: 0.9702 - val_loss: 0.1947 - val_accuracy: 0.519

In [73]:
path= '/content/download.jpg'
img = cv2.imread(path, 0)
img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
img = np.array([img]).reshape(h,w,1)/255
pred = model.predict(np.array([img]))

1/1 [==============================] - 0s 19ms/step


In [74]:
predicted_breed_index =np.argmax(pred[0])

In [75]:
breed_types = ['White Suffolk', 'Poll Dorset', 'Suffolk', 'Marino']

# Get the predicted breed based on the index
predicted_breed = breed_types[predicted_breed_index]

print("Predicted Breed:", predicted_breed)

Predicted Breed: Suffolk
